In [1]:
from openai import OpenAI
import json
import uuid
import chromadb

# create an OpenAI client
client = OpenAI()

#create a chromaDB client
chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="Real_Estate" , metadata={"hnsw:space": "cosine"} )

# Check if there is any existing data in the collection



OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
collection.count()


38

In [ ]:
# Define the function that will injects the list into the chroma DB
def generate_listing(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description):
    print(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description)
    unique_id = str(uuid.uuid4())
    collection.upsert(
    documents=[
        Description,Neighborhood_Description
    ],
    metadatas=[{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size, "Neighborhood_Description": "3"},{"Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size, "Neighborhood_Description": "3"}],
    ids=[f"{unique_id}_Description", f"{unique_id}_Neighborhood_Description"]
)


available_functions = {
        "generate_listing": generate_listing,
    }

In [ ]:
messages = [{"role": "user", "content": "create 10 sample real estate listings"}]
tools=[
    {
        "type": "function",
        "function": {
            "name": "generate_listing",
            "description": "Create a real estate Listing",
            "parameters": {
                "type": "object",
                "properties": {
                    "Neighborhood": {
                        "type": "string",
                        "description": "e.g. Green Oaks"
                    },
                    "Price": {
                        "type": "string",
                        "description": "e.g. $800,000"
                    },
                    "Bedrooms": {
                        "type": "number",
                        "description": "e.g. 2"
                    },
                    "Bathrooms": {
                        "type": "number"
                    },
                    "House Size": {
                        "type": "string",
                        "description": "e.g. 2,000 sqft"
                    },
                    "Description": {
                        "type": "string",
                        "description": "e.g. Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."
                    },
                    "Neighborhood Description": {
                        "type": "string",
                        "description": "e.g. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze"
                    }
                },
                "required": ["Neighborhood","Price","Bedrooms","Bathrooms","House Size","Description","Neighborhood Description"]
            }
        }
    }
    ]

response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
response_message = response.choices[0].message
tool_calls = response_message.tool_calls



In [ ]:
tool_calls

[ChatCompletionMessageToolCall(id='call_9tjiJsCwzYIQoPHrmDRS6WDN', function=Function(arguments='{"Neighborhood": "Green Oaks", "Price": "$800,000", "Bedrooms": 3, "Bathrooms": 2, "House Size": "2,000 sqft", "Description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.", "Neighborhood Description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy G

In [ ]:
if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                Neighborhood=function_args.get("Neighborhood"),
                Price=function_args.get("Price"),
                Bedrooms=function_args.get("Bedrooms"),
                Bathrooms=function_args.get("Bathrooms"),
                House_Size=function_args.get("House Size"),
                Description=function_args.get("Description"),
                Neighborhood_Description=function_args.get("Neighborhood Description"),
            )
            print(function_response)

Green Oaks $800,000 3 2 2,000 sqft Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze
None
Lakeview $650,000 2 1 1,500 sqft Discover urban living at its best in this stylish 2-bedroom, 1-bathroom condo located

In [ ]:
results = collection.query(
    query_texts=["high-end finishes"], # Chroma will embed this for you
    n_results=2 # how many results to return
)



In [ ]:
results

{'ids': [['9aa99797-6b99-4f2a-bd1f-45fcd12efdc3_Description',
   '18c68a11-edd9-4fa3-9c91-e48310d53a74_Description']],
 'distances': [[0.7712009623625209, 0.7996031103580203]],
 'metadatas': [[{'Bathrooms': 2,
    'Bedrooms': 3,
    'House_Size': '2,400 sqft',
    'Neighborhood': 'Gold Coast',
    'Neighborhood_Description': '3',
    'Price': '$950,000'},
   {'Bathrooms': 2,
    'Bedrooms': 3,
    'House_Size': '2,100 sqft',
    'Neighborhood': 'Old Town',
    'Neighborhood_Description': '3',
    'Price': '$820,000'}]],
 'embeddings': None,
 'documents': [['Luxury awaits in this stunning 3-bedroom, 2-bathroom condo located in the prestigious Gold Coast neighborhood. The spacious and bright interior features hardwood floors, custom millwork, and high-end finishes. The gourmet kitchen is equipped with top-of-the-line appliances and a center island. Retreat to the master suite with a luxurious en-suite bathroom and walk-in closet. Enjoy the views from the private balcony or explore the up

In [ ]:
documents = results['ids'][0]
for doc in documents:
    print(doc)

9aa99797-6b99-4f2a-bd1f-45fcd12efdc3_Description
18c68a11-edd9-4fa3-9c91-e48310d53a74_Description
